<a href="https://colab.research.google.com/github/grazh/registration_form_analysis/blob/main/DDL_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем библиотеки и считываем данные

In [245]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from datetime import datetime
from google.colab import drive
from statsmodels.stats.proportion import proportions_ztest

# drive.mount('/content/drive')
web_data = pd.read_csv('/content/drive/MyDrive/example_data.txt', sep=',')
web_data


Проверим на дубли

In [208]:
web_data[web_data.duplicated()].sort_values(by=['user_id'])
web_data = web_data.drop_duplicates()

Изучим датасет и проверим пропуски

In [209]:
def some_info(df):
    print("Some info about dataset\n") 
    
    # Columns
    print("Cloumns:\n{}".format('\n'.join(list(df.columns))))

    # Shape
    print("This dataset has {} features.".format(df.shape[1]))
    print("This dataset has {} observations.\n".format(df.shape[0]))
    
    # Describe
    print("Describe:\n", df.describe(), "\n\n")
    
    # Missing values
    print("Missing values:\n", round(df.isna().sum()/int(df.shape[0]), 2), "\n")
    
    # Data head
    print(df.head())


some_info(web_data)
print(np.unique(web_data['event']))
print(min(web_data['timestamp']))
print(max(web_data['timestamp']))

Some info about dataset

Cloumns:
user_id
timestamp
event
This dataset has 3 features.
This dataset has 1005456 observations.

Describe:
                                  user_id                      timestamp  \
count                            1005456                        1005456   
unique                            261130                         938229   
top     c9be7da3b5c975cc2795f15d45f0390a  2021-01-29 02:15:25.086335450   
freq                                5128                              9   

          event  
count   1005456  
unique        5  
top     landing  
freq     366034   


Missing values:
 user_id      0.0
timestamp    0.0
event        0.0
dtype: float64 

                            user_id                      timestamp    event
0  148870bfa84777898359aaa8e120a373  2021-01-01 00:00:01.000000000  landing
1  ac3948ea43cb39cdc4e739004d252d0b  2021-01-01 00:00:01.445020335  landing
2  48a0df50d7ed1fcaaddf742b828b85e5  2021-01-01 00:00:10.566157670    login
3  7

Убираем незарегистрированных пользователей

In [ ]:
registration_users_pre = web_data[web_data['event'].isin(['registration', 'registration_new'])]

# Определяем группу пользователя
registration_users_pre['test_control'] = registration_users_pre['event'].apply(lambda x: 'test' if x == 'registration_new' else 'control')
registration_users_pre = registration_users_pre[['user_id', 'test_control']]

# Проверяем выборки на пересечение
registration_users_pre.groupby('user_id').nunique().sort_values(by='test_control')
# Выборки не пересекаются

# Выделяем только зарегистрировавшихся клиентов
registration_users = registration_users_pre[['user_id', 'test_control']].drop_duplicates()

# Оставляем в данных только зарегистрировавшихся клиентов
clean_data = web_data.merge(registration_users, how='inner', on='user_id')


Считаем различия в количестве попыток зарегистрироваться


In [278]:
alpha_ = 0.05

test_group = clean_data[clean_data['test_control'] == 'test']
control_group = clean_data[clean_data['test_control'] == 'control']

def registration_attempts(data):
    data = data[data['event'].isin(['registration', 'registration_new'])]
    return data.groupby('user_id').count()['event']

test_attempts = registration_attempts(test_group)
control_attempts = registration_attempts(control_group)


def check_significance(test, control, metric):
    print(f"""Различия между тестом ({round(test.mean(),2)}) и контролем ({round(control.mean(),2)}) в {metric} {['' if stats.ttest_ind(test, control).pvalue < alpha_ else 'не '][0]}являются статзначимыми.
p-value: {round(stats.ttest_ind(test, control).pvalue, 2)}\n""")
  
check_significance(test_attempts, control_attempts, 'среднем количестве попыток регистрации')

Различия между тестом (2.14) и контролем (2.08) в среднем количестве попыток регистрации не являются статзначимыми.
p-value: 0.35



Считаем различия в конверсии в успешную регистрацию


In [310]:
conversion_df = clean_data.copy()
conversion_df = conversion_df.sort_values(by='user_id')
conversion_df['next_action'] = conversion_df.groupby(['user_id']).shift(-1)['event']
conversion_df.sort_values(by='user_id')

def conversion_rate(data):
    pre_result = data[data['event'].isin(['registration', 'registration_new'])].copy()
    pre_result['success'] = pre_result['next_action'].apply(lambda x: 1 if x == 'main' else 0)
    
    return pre_result.groupby('user_id')['success'].max()
  
test_conversion = conversion_rate(conversion_df[conversion_df['test_control'] == 'test'])
control_conversion = conversion_rate(conversion_df[conversion_df['test_control'] == 'control'])

check_significance(test_conversion, control_conversion, 'конверсии в успешную регистрацию')


Различия между тестом (0.29) и контролем (0.27) в конверсии в успешную регистрацию являются статзначимыми.
p-value: 0.03



Смотрим на активность пользователей

In [317]:
def compare_activity(data):
    result = data.copy()
    result['flag'] = result['event'].apply(lambda x: 1 if x in ['registration', 'registration_new'] else 0)
    result['new_flag'] = result.groupby('user_id').flag.transform(lambda x: x.cumsum())

    result = result[(result['new_flag'] > 0) & (result['flag'] == 0)]
    return result.groupby('user_id').count()['event']


check_significance(compare_activity(test_group), compare_activity(control_group), 'среднем количестве посещений разделов сайта')


Различия между тестом (11.39) и контролем (11.69) в среднем количестве посещений разделов сайта не являются статзначимыми.
p-value: 0.84

